# Notes

## Enviroments

It is important to isolate on enviroment in order to avoid the conflicts of libraries' versions.
We can use virtual enviroments for that:
* pipenv
* poetry
* conda
* ...

In [1]:
pip install pipenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pipenv]━━━━━━━━━━━ 2/3 [pipenv]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**later when you need to download the libraries into this enviroment, type 'pipenv install libraryname'**

In the "Pipfile" you can see the packaages installed:
```
[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[packages]

[dev-packages]

[requires]
python_version = "3.12"
```
-------------------------------------- 


You put all the packages to the file and then to install all listed packages, you  just type:
```
pipenv install

```
-------------------------------------- 

 With the Pipfile.lock you can  keep reproducability.

-------------------------------------- 
 To get inside and use the exact enviroment we type:
 ```
pipenv shell

```
And then we execute commands within the enviroment. Or we type pipenv run before the command:
```
$ pipenv run gunicorn --bind 0.0.0.0:9696 predict:app
```

## Containerization

If we have different system dependencies, we can get the conflict. To isolate  apps we can use containers.
Dockerhub -> python image -> does it work with debian -> docker run -it(-i for interactive and -t for TTY (pseudo-terminal). Using them together lets you interact with a container as if you were running a command in a standard shell, like a bash session.) --rm(remove after using) --entrypoint=bash (start a cli inside of docker) python:3.12-slim (name of the image)

Then we specify want we want in the Dockerfile.

```
# Base image
FROM python:3.12-slim

RUN pip install pipenv

# Create a directory and cd there
WORKDIR /app

RUN pipenv lock

# Do not create enviroment, it is already isolated
RUN pipenv install --system --deploy
RUN pip install numpy flask scikit-learn
RUN pip install gunicorn

# Copy the files from your locl machine in the build enviroment
COPY ["predict.py", "model_C=1.0.bin", "./"]
```

To build and run the container using image: 
```
docker build -t zoomcamp-test .
```
Then we can run the image we've created
```
docker run -it --rm entrypoint=bash zoomcamp-test
```
We can check if  the server is running
```
root@e883f9ebe59f:/app# gunicorn --bind=0.0.0.0:9696 predict:app
```
To expose it  to another container we need to open the port and map it to the port in the host machine.

```
# Base image
FROM python:3.12-slim

RUN pip install pipenv

# Create a directory and cd there
WORKDIR /app

RUN pipenv lock

# Do not create enviroment, it is already isolated
RUN pipenv install --system --deploy
RUN pip install numpy flask scikit-learn
RUN pip install gunicorn

# Copy the files from your locl machine in the build enviroment
COPY ["predict.py", "model_C=1.0.bin", "./"]

EXPOSE 9696

ENTRYPOINT ["gunicorn", "--bind=0.0.0.0:9696", "predict:app"]
```

We are going to run it with the entrypoint specified in the Dockerfile. We are mapping the ports "-p 9696(port from container):9696(port on the host machine)"
```
docker run -it --rm -p 9696:9696 zoomcamp-test
```

## Deploying to the AWS Beanstalk

It can help us to scale up if the number of requests to our service will rise. 
``` 
# We need it only to deploy, so it is called dev dependency, we do not need it in the container.
pipenv install awsebcli --dev

pipenv shell 
eb init -p docker -r eu-west-1 churn-serving

# test locally
eb local run --port 9696

python predict_test.py

# To run on the cloud
eb create churn-serving-env

# It creates the load manager and scaling

Then we have to change the host and port in the predict_test.py file.

```
host = 'churn-serving-env.eba-83499798.eu-west....'
url = f'http://{host}/predict
```
EB uses the defult 80 port so we do not have to specify it.
It is open to the world now. So check the permissions.

To terminate:

```
eb terminate churn-serving-env
```